In [1]:
import imaging_behavior_functions
import odor_behavioral_analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize
import scipy.io
import os
import mat73
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import interp1d
from scipy.stats import iqr
from PIL import Image
import cv2
from IPython.display import display, clear_output
import time
import seaborn as sns
from scipy.stats import iqr
from scipy.ndimage import gaussian_filter1d
import glob
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.graph_objects as go
%matplotlib inline

In [3]:
# put in path to query a specific session
base_path = "C:/Users/wilson/OneDrive - Harvard University/Thesis - Wilson lab/2P imaging/preprocessed data/qualified_sessions/one_trial_sessions/"
example_path_data = base_path+"20230805-4_MBON21hDeltaB_syntGCAMP7f_odor_apple_patchy/data/" 
example_path_results = base_path+"20230805-4_MBON21hDeltaB_syntGCAMP7f_odor_apple_patchy/results/"

trial_num = 1 # specify trial number 
odor_threshold = 5 # don't need to change just used to convert odor voltage signal to binary
time_interval_threshold = 16  # Assuming time is in seconds or an equivalent unit
k = 8 # parameter for clustering odor segments based on spatial location
window_size = 30 # window for calculating straightness 

is_mat73, roi_df, dff_raw, kinematics_raw, preprocessed_vars_ds, preprocessed_vars_odor = imaging_behavior_functions.load_intermediate_mat(example_path_data,trial_num)
behav_df = imaging_behavior_functions.make_df_behavior(dff_raw, preprocessed_vars_ds, preprocessed_vars_odor,trial_num,ball_d = 9)
xPos, yPos = imaging_behavior_functions.reconstruct_path(behav_df, ball_d = 9)
roi_names, hdeltab_index, epg_index, hdeltab_sequence, epg_sequence = imaging_behavior_functions.get_roi_seq(roi_df)
dff_all_rois, dff_time = imaging_behavior_functions.load_dff_raw(is_mat73, dff_raw)
neural_df = imaging_behavior_functions.make_df_neural(dff_all_rois, dff_time, roi_names, hdeltab_index, epg_index, hdeltab_sequence, epg_sequence)
roi_kw = 'hDeltaB'
roi_mtx = imaging_behavior_functions.extract_heatmap(neural_df, roi_kw, 1, example_path_results, trial_num)
param_df = imaging_behavior_functions.fit_sinusoid(neural_df,roi_mtx)
behav_df, padded_result_df, smoothed_df = odor_behavioral_analysis.analysis_dfs(behav_df, time_interval_threshold, k, window_size)
combined_df = imaging_behavior_functions.combine_df(behav_df, neural_df)
smoothed_df["translationalV_smoothed"] = np.sqrt(smoothed_df["fwV_smoothed"]**2+smoothed_df["sideV_smoothed"]**2)

In [4]:
# df for behavioral variables
behav_df

,time,fwV,sideV,yawV,heading,abssideV,absyawV,net_motion,net_motion_state,heading_adj,odor,xPos,yPos,cluster_label_temporal,soft_cluster_label_temporal,cluster_label_spatial
0,0.000000,0.000000,0.000000,0.000000,0.748649,0.000000,0.000000,0.000000,0,0.748649,-0.000000,0.000000,0.000000,0,0,0
1,0.240513,-0.219713,-1.306855,0.100094,0.749624,1.306855,0.100094,1.626663,1,0.749624,-0.014187,-0.017524,-0.070283,0,0,0
2,0.481026,-0.119723,-1.628911,0.059436,0.747587,1.628911,0.059436,1.808070,1,0.747587,-0.014988,-0.039228,-0.120858,0,0,0
3,0.721539,-0.303351,-2.169633,0.196950,0.750262,2.169633,0.196950,2.669934,1,0.750262,-0.015801,-0.068373,-0.222751,0,0,0
4,0.962052,0.036717,-0.108245,-0.010321,0.751260,0.108245,0.010321,0.155283,0,0.751260,-0.022401,-0.069799,-0.215363,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7480,1799.036986,10.456535,-2.430525,4.156634,5.742618,2.430525,4.156634,17.043694,1,37.158545,9.864224,467.860836,285.030287,0,0,5
7481,1799.277499,-1.282638,-16.646307,8.714621,5.963540,16.646307,8.714621,26.643566,1,37.379467,9.837771,468.218965,284.579765,0,0,5
7482,1799.518012,7.348875,-21.653753,11.722968,0.308291,21.653753,11.722968,40.725596,1,38.007403,9.833440,467.327194,285.793715,0,0,5
7483,1799.758525,4.704347,12.071842,-8.472140,0.411995,12.071842,8.472140,25.248329,1,38.111107,9.835850,467.052368,287.067115,0,0,5


In [5]:
# df for neural activity
neural_df

,time,MBON21L,MBON21R,hDeltaB_0,hDeltaB_1,hDeltaB_2,hDeltaB_3,hDeltaB_4,hDeltaB_5,hDeltaB_6,...,hDeltaB_18,hDeltaB_19,hDeltaB_20,hDeltaB_21,hDeltaB_22,hDeltaB_23,hDeltaB_24,hDeltaB_25,hDeltaB_26,hDeltaB_27
0,0.000000,0.222773,0.264867,1.241962,0.383523,0.358444,0.097757,0.011519,0.277770,0.061169,...,1.916588,2.185271,1.999191,2.162483,1.916930,1.971367,1.724920,1.422568,1.198348,1.322129
1,0.240513,0.209713,0.237666,1.122885,0.373910,0.261601,0.186200,0.079669,0.205932,0.085467,...,1.910222,2.245278,2.006683,2.076988,1.878382,2.054740,1.769468,1.327635,1.199988,1.461048
2,0.481026,0.161754,0.212745,1.138844,0.295217,0.133648,0.338941,0.132484,0.163215,0.189639,...,1.882963,2.262364,2.080175,2.052068,1.858321,2.050991,1.830000,1.274174,1.266786,1.564603
3,0.721539,0.086226,0.170872,1.136710,0.232473,0.008637,0.368918,0.121838,0.228631,0.317409,...,1.810834,2.287551,2.155690,2.116092,1.876195,1.937832,1.814542,1.178010,1.355249,1.511653
4,0.962052,0.021611,0.116444,0.784412,0.284925,-0.013781,0.299948,0.062412,0.358765,0.385596,...,1.811607,2.245671,2.199512,2.166246,1.901164,1.879810,1.712413,1.057092,1.345140,1.334321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7480,1799.036986,0.495776,0.447953,0.220035,-0.130434,0.015037,-0.042719,-0.123742,-0.077967,0.034825,...,2.678864,2.692949,2.204068,1.894932,1.306142,1.125682,0.694187,0.191132,0.156214,0.164683
7481,1799.277499,0.477877,0.464794,0.021527,-0.090348,-0.056911,-0.082641,-0.201801,-0.158782,0.034147,...,2.808143,2.674942,2.417434,2.100247,1.446980,1.464223,0.892807,0.474159,0.244738,0.307505
7482,1799.518012,0.477426,0.469343,-0.275158,-0.050738,-0.134345,-0.031096,-0.246961,-0.211673,-0.003561,...,2.950979,2.660348,2.594776,2.300975,1.788475,1.732115,1.079645,0.681985,0.223644,0.445096
7483,1799.758525,0.468498,0.453946,-0.411082,-0.063065,-0.065458,0.032444,-0.249648,-0.194095,-0.061058,...,2.937907,2.721088,2.683954,2.458006,2.080824,1.906719,1.199272,0.745784,0.225406,0.542416


In [6]:
# df for paramatric fit of neural df
param_df

,time,phase,baseline,amplitude,phase_error,baseline_error,amplitude_error
0,0.000000,4.188790,0.115561,1.478587,0.064717,0.068432,0.097852
1,0.240513,4.188790,0.165476,1.517075,0.067541,0.073189,0.104534
2,0.481026,4.188790,0.213279,1.513413,0.074924,0.080917,0.115467
3,0.721539,4.188790,0.242901,1.486588,0.081122,0.085901,0.122365
4,0.962052,3.956080,0.242280,1.480633,0.077250,0.081279,0.115510
...,...,...,...,...,...,...,...
7480,1799.036986,3.723369,0.085306,2.184039,0.036784,0.056443,0.079304
7481,1799.277499,3.723369,0.120576,2.211595,0.031433,0.049006,0.069093
7482,1799.518012,3.956080,0.195965,2.311278,0.034028,0.055570,0.078528
7483,1799.758525,3.956080,0.283633,2.411280,0.041650,0.071020,0.100446


In [7]:
# mostly same as behav_df but added columns of smoothed velocity variables
smoothed_df

,time,fwV,sideV,yawV,heading,abssideV,absyawV,net_motion,net_motion_state,heading_adj,...,yPos,cluster_label_temporal,soft_cluster_label_temporal,cluster_label_spatial,fwV_smoothed,sideV_smoothed,yawV_smoothed,heading_smoothed,straightness_smoothed,translationalV_smoothed
0,0.000000,0.000000,0.000000,0.000000,0.748649,0.000000,0.000000,0.000000,0,0.748649,...,0.000000,0,0,0,-0.108750,-0.883365,0.059913,0.749072,0.750428,0.890034
1,0.240513,-0.219713,-1.306855,0.100094,0.749624,1.306855,0.100094,1.626663,1,0.749624,...,-0.070283,0,0,0,-0.099858,-0.957417,0.074882,0.749577,0.727487,0.962611
2,0.481026,-0.119723,-1.628911,0.059436,0.747587,1.628911,0.059436,1.808070,1,0.747587,...,-0.120858,0,0,0,-0.048625,-1.056604,0.124079,0.751587,0.729715,1.057722
3,0.721539,-0.303351,-2.169633,0.196950,0.750262,2.169633,0.196950,2.669934,1,0.750262,...,-0.222751,0,0,0,0.110787,-1.228274,0.266880,0.758248,0.736050,1.233261
4,0.962052,0.036717,-0.108245,-0.010321,0.751260,0.108245,0.010321,0.155283,0,0.751260,...,-0.215363,0,0,0,0.463820,-1.706959,0.604848,0.776593,0.700775,1.768852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7480,1799.036986,10.456535,-2.430525,4.156634,5.742618,2.430525,4.156634,17.043694,1,37.158545,...,285.030287,0,0,5,6.681701,-7.481783,2.490296,5.920101,NaN,10.031062
7481,1799.277499,-1.282638,-16.646307,8.714621,5.963540,16.646307,8.714621,26.643566,1,37.379467,...,284.579765,0,0,5,5.981866,-7.410167,2.610321,6.067336,NaN,9.523303
7482,1799.518012,7.348875,-21.653753,11.722968,0.308291,21.653753,11.722968,40.725596,1,38.007403,...,285.793715,0,0,5,5.448869,-6.225567,1.675990,6.204412,NaN,8.273322
7483,1799.758525,4.704347,12.071842,-8.472140,0.411995,12.071842,8.472140,25.248329,1,38.111107,...,287.067115,0,0,5,5.200999,-4.295300,0.056905,0.009931,NaN,6.745369


In [8]:
# analysis of trajectory stats like curvature, straightness etc. by moving window 
padded_result_df

,Curvature_Mean,Curvature_Std,Tight_Turns,Moderate_Turns,Wide_Turns,Heading_Variance,Mean_fwV_sideV_Ratio,Yaw_Velocity_Mean,Yaw_Velocity_Std
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
